<a href="https://colab.research.google.com/github/madhusudanhasbe/Natural_Language_Processing_Lab/blob/main/22070126061_NLP_Assignment_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>